In [1]:
import numpy as np
import open3d as o3d

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


Here we use pointcloud from [semantic_spray_dataset](https://github.innominds.com/aldipiroli/semantic_spray_dataset). You can download it via the [link](https://oparu.uni-ulm.de/xmlui/handle/123456789/48891).

Load pointcloud and it's labels

In [2]:
cloud_id = "000050"
points = np.fromfile(f"{cloud_id}.bin", np.float32).reshape(-1, 5)  # x,y,z,intensity,ring
labels = np.fromfile(f"{cloud_id}.label", np.int32).reshape(-1)  # 0: background, 1: foreground (vehicle), 2: noise
print(points.shape, labels.shape)

(30422, 5) (30422,)


Create colors array for each label

In [3]:
colors = np.zeros((points.shape[0], 3))
colors[labels == 0, 0] = 1. # background - red
colors[labels == 1, 1] = 1. # foreground (vehicle) - green
colors[labels == 2, 2] = 1. # noise - blue

Create Open3D pointcloud 

In [4]:
pcd = o3d.geometry.PointCloud()
pcd.points = o3d.utility.Vector3dVector(points[:, :3])
pcd.colors = o3d.utility.Vector3dVector(colors)

Visualize cloud

In [5]:
o3d.visualization.draw_geometries([pcd])

### TODO

Create model to classify each point in three categories:
- 0: background
- 1: foreground (vehicle)
- 2: noise  

Hints:
- you can use points located only in the near area (e.g. x in [-20; 20], y in [-20, 20])
- you can filter out ground surface using z-threshold, RANSAC or other algorithm
- instead of points you can clusterize them and classify clusters

In [3]:
import numpy as np
import open3d as o3d
import matplotlib.pyplot as plt

cloud_id = "000050"
points = np.fromfile(f"{cloud_id}.bin", np.float32).reshape(-1, 5)  # x,y,z,intensity,ring
labels = np.fromfile(f"{cloud_id}.label", np.int32).reshape(-1)  # 0: background, 1: foreground (vehicle), 2: noise
print(points.shape, labels.shape)

colors = np.zeros((points.shape[0], 3))
colors[labels == 0, 0] = 1. # background - red
colors[labels == 1, 1] = 1. # foreground (vehicle) - green
colors[labels == 2, 2] = 1. # noise - blue

pcd = o3d.geometry.PointCloud()
pcd.points = o3d.utility.Vector3dVector(points[:, :3])
pcd.colors = o3d.utility.Vector3dVector(colors)

(30422, 5) (30422,)


In [21]:
pcd_center = pcd.get_center()
pcd.translate(-pcd_center)

PointCloud with 30422 points.

In [37]:
# Statistical outliers filter
nn = 10 
std_multiplier = 10
filtered_pcd = pcd.remove_statistical_outlier(nn, std_multiplier)
outliers = pcd.select_by_index(filtered_pcd[1], invert=True)
outliers.paint_uniform_color([0, 0, 0]) # black - outliers
filtered_pcd = filtered_pcd[0]
o3d.visualization.draw_geometries([filtered_pcd, outliers])

In [38]:
# Voxel downsampling
voxel_size = 0.01
pcd_downsampled = filtered_pcd.voxel_down_sample(voxel_size=voxel_size)
o3d.visualization.draw_geometries([pcd_downsampled])

In [39]:
nn_distance = np.mean(pcd.compute_nearest_neighbor_distance())
radius_normals = nn_distance * 4
pcd_downsampled.estimate_normals(search_param=o3d.geometry.KDTreeSearchParamHybrid(radius=radius_normals, max_nn=16), fast_normal_computation=True)
o3d.visualization.draw_geometries([pcd_downsampled, outliers])

In [47]:
# RANSAC Planar Segmentation

pt_to_plane_dist = 0.6

plane_model, inliers = pcd.segment_plane(distance_threshold=pt_to_plane_dist, ransac_n=3, num_iterations=1000)
[a, b, c, d] = plane_model

inlier_cloud = pcd.select_by_index(inliers)
outlier_cloud = pcd.select_by_index(inliers, invert=True)

inlier_cloud.paint_uniform_color([0.6, 0.6, 0.6]) 
outlier_cloud.paint_uniform_color([0, 0, 0]) 

o3d.visualization.draw_geometries([inlier_cloud, outlier_cloud])

In [23]:
import numpy as np
import open3d as o3d
import matplotlib.pyplot as plt

cloud_id = "000050"
points = np.fromfile(f"{cloud_id}.bin", np.float32).reshape(-1, 5)  # x,y,z,intensity,ring
labels = np.fromfile(f"{cloud_id}.label", np.int32).reshape(-1)  # 0: background, 1: foreground (vehicle), 2: noise
print(points.shape, labels.shape)

colors = np.zeros((points.shape[0], 3))
colors[labels == 0, 0] = 1. # background - red
colors[labels == 1, 1] = 1. # foreground (vehicle) - green
colors[labels == 2, 2] = 1. # noise - blue

pcd = o3d.geometry.PointCloud()
pcd.points = o3d.utility.Vector3dVector(points[:, :3])
pcd.colors = o3d.utility.Vector3dVector(colors)

(30422, 5) (30422,)


In [24]:
import copy

def draw_registration_result(source, target, transformation):
    source_temp = copy.deepcopy(source)
    target_temp = copy.deepcopy(target)
    source_temp.paint_uniform_color([1, 0.706, 0])
    target_temp.paint_uniform_color([0, 0.651, 0.929])
    source_temp.transform(transformation)
    o3d.visualization.draw_geometries([source_temp, target_temp],
                                      zoom=0.4559,
                                      front=[0.6452, -0.3036, -0.7011],
                                      lookat=[1.9892, 2.0208, 1.8945],
                                      up=[-0.2779, -0.9482, 0.1556])

def preprocess_point_cloud(pcd, voxel_size):
    print(":: Downsample with a voxel size %.3f." % voxel_size)
    pcd_down = pcd.voxel_down_sample(voxel_size)

    radius_normal = voxel_size * 2
    print(":: Estimate normal with search radius %.3f." % radius_normal)
    pcd_down.estimate_normals(
        o3d.geometry.KDTreeSearchParamHybrid(radius=radius_normal, max_nn=30))

    radius_feature = voxel_size * 5
    print(":: Compute FPFH feature with search radius %.3f." % radius_feature)
    pcd_fpfh = o3d.pipelines.registration.compute_fpfh_feature(
        pcd_down,
        o3d.geometry.KDTreeSearchParamHybrid(radius=radius_feature, max_nn=100))
    return pcd_down, pcd_fpfh

def prepare_dataset(voxel_size):
    print(":: Load two point clouds and disturb initial pose.")

    demo_icp_pcds = o3d.data.DemoICPPointClouds()
    source = o3d.io.read_point_cloud(demo_icp_pcds.paths[0])
    target = o3d.io.read_point_cloud(demo_icp_pcds.paths[1])
    trans_init = np.asarray([[0.0, 0.0, 1.0, 0.0], [1.0, 0.0, 0.0, 0.0],
                             [0.0, 1.0, 0.0, 0.0], [0.0, 0.0, 0.0, 1.0]])
    source.transform(trans_init)
    draw_registration_result(source, target, np.identity(4))

    source_down, source_fpfh = preprocess_point_cloud(source, voxel_size)
    target_down, target_fpfh = preprocess_point_cloud(target, voxel_size)
    return source, target, source_down, target_down, source_fpfh, target_fpfh

voxel_size = 0.05  # means 5cm for this dataset
source, target, source_down, target_down, source_fpfh, target_fpfh = prepare_dataset(voxel_size)

:: Load two point clouds and disturb initial pose.
[Open3D INFO] Downloading https://github.com/isl-org/open3d_downloads/releases/download/20220301-data/DemoICPPointClouds.zip
[Open3D INFO] Downloaded to C:\Users\alexuserx/open3d_data/download/DemoICPPointClouds/DemoICPPointClouds.zip
[Open3D INFO] Created directory C:\Users\alexuserx/open3d_data/extract/DemoICPPointClouds.
[Open3D INFO] Extracting C:\Users\alexuserx/open3d_data/download/DemoICPPointClouds/DemoICPPointClouds.zip.
[Open3D INFO] Extracted to C:\Users\alexuserx/open3d_data/extract/DemoICPPointClouds.
:: Downsample with a voxel size 0.050.
:: Estimate normal with search radius 0.100.
:: Compute FPFH feature with search radius 0.250.
:: Downsample with a voxel size 0.050.
:: Estimate normal with search radius 0.100.
:: Compute FPFH feature with search radius 0.250.


In [25]:
def execute_global_registration(source_down, target_down, source_fpfh,
                                target_fpfh, voxel_size):
    distance_threshold = voxel_size * 1.5
    print(":: RANSAC registration on downsampled point clouds.")
    print("   Since the downsampling voxel size is %.3f," % voxel_size)
    print("   we use a liberal distance threshold %.3f." % distance_threshold)
    result = o3d.pipelines.registration.registration_ransac_based_on_feature_matching(
        source_down, target_down, source_fpfh, target_fpfh, True,
        distance_threshold,
        o3d.pipelines.registration.TransformationEstimationPointToPoint(False),
        3, [
            o3d.pipelines.registration.CorrespondenceCheckerBasedOnEdgeLength(
                0.9),
            o3d.pipelines.registration.CorrespondenceCheckerBasedOnDistance(
                distance_threshold)
        ], o3d.pipelines.registration.RANSACConvergenceCriteria(100000, 0.999))
    return result

In [26]:
result_ransac = execute_global_registration(source_down, target_down,
                                            source_fpfh, target_fpfh,
                                            voxel_size)
print(result_ransac)
draw_registration_result(source_down, target_down, result_ransac.transformation)

:: RANSAC registration on downsampled point clouds.
   Since the downsampling voxel size is 0.050,
   we use a liberal distance threshold 0.075.
RegistrationResult with fitness=6.680672e-01, inlier_rmse=3.688161e-02, and correspondence_set size of 3180
Access transformation to get result.


In [27]:
def refine_registration(source, target, source_fpfh, target_fpfh, voxel_size):
    distance_threshold = voxel_size * 0.4
    print(":: Point-to-plane ICP registration is applied on original point")
    print("   clouds to refine the alignment. This time we use a strict")
    print("   distance threshold %.3f." % distance_threshold)
    result = o3d.pipelines.registration.registration_icp(
        source, target, distance_threshold, result_ransac.transformation,
        o3d.pipelines.registration.TransformationEstimationPointToPlane())
    return result

In [28]:
result_icp = refine_registration(source, target, source_fpfh, target_fpfh,
                                 voxel_size)
print(result_icp)
draw_registration_result(source, target, result_icp.transformation)

:: Point-to-plane ICP registration is applied on original point
   clouds to refine the alignment. This time we use a strict
   distance threshold 0.020.
RegistrationResult with fitness=6.210275e-01, inlier_rmse=6.565172e-03, and correspondence_set size of 123482
Access transformation to get result.


In [29]:
voxel_size = 0.05  # means 5cm for the dataset
source, target, source_down, target_down, source_fpfh, target_fpfh = \
        prepare_dataset(voxel_size)

:: Load two point clouds and disturb initial pose.
:: Downsample with a voxel size 0.050.
:: Estimate normal with search radius 0.100.
:: Compute FPFH feature with search radius 0.250.
:: Downsample with a voxel size 0.050.
:: Estimate normal with search radius 0.100.
:: Compute FPFH feature with search radius 0.250.


In [30]:
import time

start = time.time()
result_ransac = execute_global_registration(source_down, target_down,
                                            source_fpfh, target_fpfh,
                                            voxel_size)
print("Global registration took %.3f sec.\n" % (time.time() - start))
print(result_ransac)
draw_registration_result(source_down, target_down, result_ransac.transformation)

:: RANSAC registration on downsampled point clouds.
   Since the downsampling voxel size is 0.050,
   we use a liberal distance threshold 0.075.
Global registration took 0.071 sec.

RegistrationResult with fitness=6.750000e-01, inlier_rmse=3.660693e-02, and correspondence_set size of 3213
Access transformation to get result.
